In [ ]:
%load_ext autoreload
%autoreload 2
import os
if 'Notebooks' in os.getcwd():
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    os.chdir(project_root)
from Modules.ModelModule import DataPreparation
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from Modules.DataLoaderModule import DataLoader
from Modules.PreprocessingModule import Preprocessor
from keras_tuner import Hyperband

In [2]:
loader = DataLoader()
X, Y = loader.load_data_from_folder_as_png('data')
X_train, X_test, X_val, Y_train, Y_test, Y_val = Preprocessor.split_data(X, Y)


X_train, Y_train = DataPreparation.data_formatting(
     Preprocessor.preprocessing(X_train, Y_train)
)

X_val, Y_val = DataPreparation.data_formatting(
    Preprocessor.preprocessing(X_val, Y_val)
)
X_test, Y_test = DataPreparation.data_formatting(
    Preprocessor.preprocessing(X_test, Y_test)
)


In [10]:
def build_model(hp):
    kernel_size_1 = hp.Choice('kernel_size_1', [3, 5, 7])
    kernel_size_2 = hp.Choice('kernel_size_2', [3, 5, 7]) 
    model = Sequential([
        Input(shape=(224, 224, 1)),
        Conv2D(
            filters=hp.Int('conv_1_filters', min_value=16, max_value=64, step=16), 
            kernel_size=(kernel_size_1, kernel_size_1),
            activation='relu'
        ),
        MaxPooling2D((2, 2)),
        Conv2D(
            filters=hp.Int('conv_2_filters', min_value=32, max_value=128, step=32), 
            kernel_size=(kernel_size_2, kernel_size_2), 
            activation='relu'
        ),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(
            units=hp.Int('dense_units', min_value=32, max_value=128, step=32), 
            activation='relu'
        ),
        Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', 'AUC', 'Precision', 'Recall']
    )
    return model



In [ ]:
tuner = Hyperband(
    build_model,
    objective='val_loss',   
    max_epochs=20,              
    factor=3,                   
    directory='ModelTests',
    project_name='cnn_complex'  
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


tuner.search(
    X_train, Y_train,                    
    validation_data=(X_val, Y_val), 
    callbacks=[stop_early]          
)